# Import Libraries

In [2]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [3]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments



# Import Dataset

In [4]:
# Load and preprocess the data
df_fine_tuning = pd.read_csv('train.csv')
df_fine_tuning = df_fine_tuning.dropna()

df_fine_tuning = df_fine_tuning.dropna().apply(lambda x: x.str.strip())
df_fine_tuning = df_fine_tuning.sample(n=20, random_state=42)
# Shuffle and split the dataset
train_data = df_fine_tuning.sample(frac=0.8, random_state=42)
eval_data = df_fine_tuning.drop(train_data.index)





In [18]:
train_data.head()

,Context,Response
3155,We've been in a long distance relationship for two and a half years. I recently saw his phone and saw the people he texts the most and one of them was a female coworker. I don't know how to approach this situation. How do I ask him about it?.,"If you'd like to ask a question, then go ahead and ask!Boyfriend/girlfriend is a close relationship and it is usually understood as an exclusive relationship. You're definitely entitled to know if your wishes to not have him texting another woman, are being respected.Often people are afraid to ask because they fear the truth will hurt them.In the short term this is definitely true.In the long term, knowing you are getting what you want and at the very least stating your expectations to your boyfriend, will clarify for him, what is meaningful in your relationship."
864,My fiancé and I broke up. He cheated on me numerous times. I kept forgiving but questioning his every move. He got tired and left.,"Cheating on you shows a lack of commitment, in addition to the emotional hurt it creates.Probably you didn't actually forgive him because if you did, then you wouldn't have been asking questions of his every move.Maybe you were open to forgiving him. In order for forgiveness to be effective, the person who has done the injuring must first show some understanding and empathy for the great pain the person caused in you.From what you write, your fiancé didn't seem to have much interest in earning back your trust or in empathizing with the way his cheating effected you.It is very likely his tiredness is also tiredness you feel, of having to watch him all the time.As uncomfortable as adjusting with the disappointment of him leaving you, the situation you describe sounds like it was unsatisfying for both of you."
3256,Does counseling really do anything that can help people?,"I believe that counseling (and finding a GOOD therapist)\r\nwill be of significant help when the individual seeking therapy desires change,\r\nimprovement, and overall growth. Most counseling involves homework and a level\r\nof commitment outside of session that also contributes to resolution of\r\nproblems and an ability to cope with stresses in a productive way. Here are\r\nsome of the key ways that therapy can help. \n1. Helps to define the problem and gain a clearer\r\nunderstanding of where it originates. Many people seek counseling because of symptoms they are\r\nexperiencing. These can include feeling sad, increased sleeping, substance use,\r\npanic attacks, self-harm, isolating one’s self, academic decline, etc. Often\r\ntimes, people are not aware of what is causing these symptoms. Having a\r\ntherapist to help guide and explore why the symptoms are occurring and where\r\nthey come from will give individuals increased insight and awareness into their\r\nproblems. \n2. Allo..."
315,"I have no real friends. I have a girlfriend who irritates me but loves me to death. I push her away and pushes me away. We’re going through a breakup, and I have nobody.","Having time all to yourself may be necessary so you have time to reflect on your own identity and values, become more clear on what matters most to you, and with this increased self-understanding, be able to attract people who will feel like satisfying friends.Having a relationship with someone whom you push away and are pushed away, with someone who is irritating, seems to have limited benefit for each of you.Maybe you are together from fear of being alone, and being alone is exactly what you may now need in your life in order to draw more favorable people to you."
506,"I’ve been on 0.5 mg of Xanax twice a day for the past month. It hasn't been helping me at all, but when I take 1 mg during a big anxiety attack, it calms me down. I was wondering how I can ask my psychologist to up the dose to 1 mg twice a day without her thinking I'm abusing them. I just have very big anxiety attacks. Should I stay on the 0.5mg and deal with the attacks or should I as

In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
# Set the eos_token as the pad_token
tokenizer.pad_token = tokenizer.eos_token

C:\Users\tashf\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\tashf\anaconda3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
# Define the custom dataset class
class ChatDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_max_length=512, target_max_length=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_max_length = source_max_length
        self.target_max_length = target_max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        source_text = row['Context']
        target_text = row['Response']

        # Tokenize and encode the source and target texts
        source_encoding = self.tokenizer(
            source_text,
            max_length=self.source_max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer(
            target_text,
            max_length=self.target_max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        labels = target_encoding['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100  # Ignore padding in loss calculation

        return {
            'input_ids': source_encoding['input_ids'].flatten(),
            'attention_mask': source_encoding['attention_mask'].flatten(),
            'labels': labels.flatten()
        }

# Create datasets
train_dataset = ChatDataset(train_data, tokenizer)
eval_dataset = ChatDataset(eval_data, tokenizer)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=2)

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=2)


In [8]:
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=10,              # Total number of training epochs
    per_device_train_batch_size=5,   # Batch size per device during training
    per_device_eval_batch_size=5,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    report_to="none",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_total_limit=2
)

trainer = Trainer(
    model=model,                         # The pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=eval_dataset            # Evaluation dataset
)


trainer.train()


c:\Users\tashf\anaconda3\lib\site-packages\accelerate\accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 12.897117614746094, 'eval_runtime': 24.4133, 'eval_samples_per_second': 0.164, 'eval_steps_per_second': 0.041, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 9.166402816772461, 'eval_runtime': 24.7115, 'eval_samples_per_second': 0.162, 'eval_steps_per_second': 0.04, 'epoch': 9.0}
{'loss': 9.7787, 'grad_norm': 22.05019760131836, 'learning_rate': 4.000000000000001e-06, 'epoch': 10.0}


  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 8.789209365844727, 'eval_runtime': 25.2268, 'eval_samples_per_second': 0.159, 'eval_steps_per_second': 0.04, 'epoch': 10.0}
{'train_runtime': 6650.1843, 'train_samples_per_second': 0.024, 'train_steps_per_second': 0.006, 'train_loss': 11.386017417907714, 'epoch': 10.0}


TrainOutput(global_step=40, training_loss=11.386017417907714, metrics={'train_runtime': 6650.1843, 'train_samples_per_second': 0.024, 'train_steps_per_second': 0.006, 'train_loss': 11.386017417907714, 'epoch': 10.0})

In [9]:
# Save the fine-tuned model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')


('./fine_tuned_model\\tokenizer_config.json',
 './fine_tuned_model\\special_tokens_map.json',
 './fine_tuned_model\\vocab.json',
 './fine_tuned_model\\merges.txt',
 './fine_tuned_model\\added_tokens.json',
 './fine_tuned_model\\tokenizer.json')

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the fine-tuned model and tokenizer from the saved directory
tokenizer = AutoTokenizer.from_pretrained('./fine_tuned_model', padding_side='left')
model = AutoModelForCausalLM.from_pretrained('./fine_tuned_model')

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))



A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm good, how are you?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: That's good


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure what to do


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm sorry


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I can't help you
